In [10]:
import helper_fxns as hf
import transforms as tr
import importlib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from pylab import get_cmap
import requests
import sys
from skimage.draw import polygon
import cv2
import copy
import math
import pandas as pd

In [2]:
import random

In [13]:
with open("train_list.txt", "r") as f:
    vois = [x.split(',') for x in f.read().split("\n")]

In [23]:
voi_df = pd.DataFrame(vois, columns = ["Filename", "x1", "x2", "y1", "y2", "z1", "z2", "cls"]).dropna()
voi_df = voi_df.astype({"x1": int, "x2": int, "y1": int, "y2": int, "z1": int, "z2": int})

In [55]:
def extract_voi(img, vois, final_size = [64,64,16], aug=False):
    voi_imgs = []
    classes = []
    
    if aug:
        crop_size = [math.ceil(x) for x in final_size*math.sqrt(2)]
    else:
        crop_size = final_size
    
    for _, voi in vois.iterrows():
        xpad = crop_size[0] - (voi['x2']-voi['x1'])
        ypad = crop_size[1] - (voi['y2']-voi['y1'])
        zpad = crop_size[2] - (voi['z2']-voi['z1'])
        pad = (-math.ceil(xpad/2), math.floor(xpad/2),
               -math.ceil(ypad/2), math.floor(ypad/2),
               -math.ceil(zpad/2), math.floor(zpad/2))

        for i in range(6):
            voi[i+1] += pad[i]
            
        if aug:
            return None
            base_img = img[voi['x1']:voi['x2'], voi['y1']:voi['y2'], voi['z1']:voi['z2']]
            aug_imgs = aug_rotation(base_img, final_size)
            voi_imgs = voi_imgs + aug_imgs
            classes.append(voi['cls'] * len(aug_imgs))
            
        else:
            voi_imgs.append(img[voi['x1']:voi['x2'], voi['y1']:voi['y2'], voi['z1']:voi['z2']])
            classes.append(voi['cls'])
    
    return voi_imgs, classes

In [64]:
for img_fn in os.listdir("imgs\\"):
    img = np.load("imgs\\"+img_fn)
    cropped_imgs, classes = extract_voi(img, voi_df[voi_df["Filename"] == img_fn])
    
    for cl in set(classes):
        subdir = "train_imgs\\"+cl
        if not os.path.exists(subdir):
            os.makedirs(subdir)
            
    for i in range(len(classes)):
        np.save("train_imgs\\"+classes[i]+"\\"+img_fn[:-4]+"_"+str(i), cropped_imgs[i])

In [52]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in xrange(0, image.shape[0], stepSize):
        for x in xrange(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [40]:
def aug_rotation(img, num_samples = 5):
    aug_imgs = [img]
    
    for _ in range(num_samples):
        angle = int(random.uniform(0, 365))
        aug_imgs.append(tr.rotate(img, angle))
        for roi in mrn_rois:
            aug_rois.append([mrn+"_r"+str(angle)+".npy"] + roi[1:])
    
    return aug_imgs

In [ ]:
with open('aug_train_list.txt', 'w') as f:
    for line in aug_lines:
        f.write(",".join(roi) + "\n")